In [ ]:
import tensorflow as tf
from jupyterUtils import *
from sklearn.metrics import classification_report
import pandas as pd
import seaborn as sns

In [ ]:
progress_bar = ProgressBar(message="Loading Model")
model_path   = '../models/10000_images_2_layers'
model        = tf.keras.models.load_model(model_path)
progress_bar.stop()
model.summary()

In [ ]:
progress_bar = ProgressBar(message="Creating Validaton Dataset")

validation_dataset = create_normalized_dataset(
    dataset_dir      = '../data/images',
    validation_split = 0.2,
    subset           = "validation",
    seed             = 123,
    image_size       = (224, 224),
    batch_size       = 32
)
progress_bar.stop()


In [ ]:
progress_bar = ProgressBar(message="Evaluating Validation Set")

true, predicted = evaluate_dataset(model, validation_dataset)

progress_bar.stop()

In [ ]:
image_path   = "../data/images/moth/moth_9901.jpg"
progress_bar = ProgressBar(message="Classifying Image")
predict_image(model, image_path, validation_dataset.class_names, 80)
progress_bar.stop()

In [ ]:
cm = confusion_matrix(true, predicted)

# Display the confusion matrix using a heatmap
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=validation_dataset.class_names).plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Generate the classification report as a dictionary
report = classification_report(true, predicted, target_names=validation_dataset.class_names, output_dict=True)

# Convert to DataFrame for easier visualization
report_df = pd.DataFrame(report).transpose()

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(report_df.iloc[:-1, :-1], annot=True, cmap='YlGnBu', cbar=False)
plt.title("Classification Report Heatmap")
plt.show()
